# Modeling

In this section, we will setup a Docker with MLFLOW and Postgres, to keep track of the models trained.  
https://github.com/aganse/docker_mlflow_db

First, we will try to overfit a few models with a selection of features, to make a sanity check on weather it's possible to train a model to a reasonable degree.  
